In [1]:
import h3
from bitarray import bitarray
from bitarray.util import ba2int, ba2hex
import toolz

def to_lower(h):
    d = h3.string_to_h3(h)
    b = '{0:064b}'.format(d)

    return -ba2int(bitarray(b)[-52:])

def rep(h):
    d = h3.string_to_h3(h)
    b = '{0:064b}'.format(d)[-52:]
    
    return b



def child_seq(h):
    """ Return list of child choices.
    """
    d = h3.string_to_h3(h)
    b = '{0:064b}'.format(d)[-45:]
    
    g = toolz.partition(3, b)
    g = (''.join(e) for e in g)
    g = map(bitarray, g)
    g = map(ba2int, g)
    g = filter(lambda x: x != 7, g)
    g = list(g)
    
    return g
    
def descr(h):
    return h3.h3_get_base_cell(h), child_seq(h)


# don't need the working set. we can work in-place!


def is_child(p, c):
    r = h3.h3_get_resolution(p)
    return p == h3.h3_to_parent(c, r)

def do_it(seq):
    cur_parent = []
    cur_children = []
    working = []

    seq = list(reversed(list(seq)))
    
    while seq:
        e = seq.pop()
        p = h3.h3_to_parent(e)
            
        if not cur_parent:
            cur_parent = [p]
            cur_children = [1]
            working = [e]
            continue

        if cur_parent[-1] == p:
            working.append(e)
            cur_children[-1] += 1
            if cur_children[-1] == 7:
                p = cur_parent.pop()
                cur_children.pop()
                for _ in range(7):
                    working.pop()
                seq.append(p)
        elif is_child(cur_parent[-1], e):
            cur_parent.append(p)
            cur_children.append(1)
            working.append(e)
        else:
            for w in working:
                yield w
            cur_parent = []
            cur_children = []
            working = []
            seq.append(e)
            
    for w in working:
        yield w

In [35]:
## single resolution

h = h3.geo_to_h3(0,0,2)

hexes = h3.h3_to_children(h, 8)

seq = list(sorted(hexes, key=to_lower, reverse=True))

In [36]:
out = list(do_it(seq))
out

['82754ffffffffff']

In [37]:
def do_it2(hexes):
    seq = list(sorted(hexes, key=to_lower, reverse=True))
    N = len(seq)
    
    num_children = []
    
    i = 0 # dense to the left of i. stack
    j = 0 # to inspect to the right of j. stream
    
    while j < N:
        if seq[j] == 0:
            j += 1
            continue
        
        # get the current working parent
        if i == 0 or h3.h3_get_resolution(seq[i-1]) == 0:
            cur_parent = None
        else:
            cur_parent = h3.h3_to_parent(seq[i-1])
        
        # pull off of upper stream. we don't have to put it back usually, can just peek.
        e = seq[j]
        j += 1
        p = h3.h3_to_parent(e)
        
        if cur_parent is None:
            #cur_parent = p
            num_children.append(1) # same as = [1]. not if we hit a top level cell! or, actuallyy...
            
            # pop onto lower stack
            seq[i] = e
            i += 1
            continue
            
        if cur_parent == p:
            num_children[-1] += 1
            seq[i] = e
            i += 1
            if num_children[-1] == 7:
                num_children.pop()
                i -= 7
                j -= 1
                seq[j] = p
            continue
        
        if is_child(cur_parent, e):
            num_children.append(1)

            # pop onto lower stack
            seq[i] = e
            i += 1
            continue
        else:
            num_children = []
            j -= 1
            seq[j] = e
            
    
    return seq[:i]

In [38]:
do_it2(hexes)

['82754ffffffffff']